In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics import roc_auc_score
from urllib import request

In [2]:
x_test = pd.read_csv('x_test.csv')
y_test = pd.read_csv('y_test.csv')

In [3]:
x_test.head(4)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,6.209642,195.417721,13116.707712,7.879348,333.699670,417.241656,7.507662,70.547387,3.612520
1,7.039094,179.645225,28827.360186,4.945555,389.889271,593.396225,12.079207,58.363507,4.366031
2,6.745307,218.424637,21871.838700,7.165509,333.699670,373.050102,16.826619,74.086080,3.902353
3,4.979469,240.813475,24167.842882,5.593830,333.699670,398.028961,6.091227,62.419529,3.439022


#### Проверка работоспособности API

In [4]:
def get_prediction(x):
    ph, hardness, solids, chloramines, sulfate = x
    body = {'ph': ph, 'Hardness': hardness, 'Solids': solids,
            'Chloramines': chloramines, 'Sulfate': sulfate}

    myurl = 'http://127.0.0.1:8180/predict'
    req = request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')
    req.add_header('Content-Length', len(jsondataasbytes))
    response = request.urlopen(req, jsondataasbytes)
    
    return json.loads(response.read())['predictions']

In [5]:
predictions = x_test[['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate']].iloc[:500].apply(lambda x: get_prediction(x), 1)

In [6]:
predictions

0      0.34
1      0.31
2      0.43
3      0.78
4      0.50
       ... 
495    0.32
496    0.35
497    0.31
498    0.39
499    0.28
Length: 500, dtype: float64

In [7]:
# Проверим roc_auc_score:

roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.7608562030699714

#### Примеры

In [8]:
# Все данные введены корректно:

get_prediction(("124.436", "574.12341", "231.231231", "5745.232", "46.123123"))

0.67

In [9]:
# Среди значений есть пустое:

get_prediction(("", "574.12341", "231.231231", "5745.232", "46.123123"))

0.68

In [10]:
# Среди значений есть пробелы, буквы и знаки препинания:

get_prediction(("sd", " ", ".", "123", "!Sdss_"))

0.55